In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from tensorflow import keras
from math import sqrt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import csv

In [3]:
#define functions for calculating auroc and auprc and 95% CI's
def roc_auc_ci(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower, AUC, upper)
def roc_prc_ci(y_true, y_score, positive=1):
    AUC = average_precision_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower, AUC, upper)

In [4]:
#read in the training and test data
data = pd.read_feather(r'/media/kchen/2TB/kchen_backup/readm/data/procol_train.feather')
y = data['READMISSION1']
X = data.drop(['READMISSION1','CASEID'], axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
test = pd.read_feather(r'/media/kchen/2TB/kchen_backup/readm/data/procol_test.feather')
y_test = test['READMISSION1']
X_test = test.drop(['READMISSION1','CASEID'], axis=1)

In [5]:
#input shape is the shape of the training data
input_shape = [X_train.shape[1]]
#define NN model based on parameters from search_keras.py
model4 = keras.models.Sequential()
model4.add(keras.layers.Flatten(input_shape=input_shape))
model4.add(keras.layers.BatchNormalization())
for _ in range(2):
    model4.add(keras.layers.Dense(1000))
    model4.add(keras.layers.BatchNormalization())
    model4.add(keras.layers.Dropout(0.8))
    model4.add(keras.layers.Activation("relu"))
model4.add(keras.layers.Dense(1, activation="sigmoid"))

opt = keras.optimizers.Adam(learning_rate=3e-3)

metrics = [keras.metrics.Recall(name='Sensitivity'), keras.metrics.TrueNegatives(name='tn'), keras.metrics.AUC(name='auc'), keras.metrics.AUC(name='prc', curve='PR')]

model4.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    metrics=metrics,)

early_stopping = keras.callbacks.EarlyStopping(
    patience=25,
    min_delta=1e-6,
    restore_best_weights=True,)

history = model4.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping],)
#calculate the auroc and auprc for the test set
ann_preds_readm = model4.predict(X_test)
ann_auc_readm = roc_auc_ci(y_test, ann_preds_readm)
ann_prc_readm = roc_prc_ci(y_test, ann_preds_readm)
#store the tpr/fpr and precision/recall for curves
ann_fpr_readm, ann_tpr_readm, _ = roc_curve(y_test, ann_preds_readm)
%store ann_fpr_readm
%store ann_tpr_readm
ann_prec_readm, ann_rec_readm, _ = precision_recall_curve(y_test, ann_preds_readm)
%store ann_prec_readm
%store ann_rec_readm

2022-01-12 13:35:48.778132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 13:35:48.784767: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 13:35:48.785272: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 13:35:48.785825: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/200
335/335 [==============================] - 3s 6ms/step - loss: 0.3709 - Sensitivity: 0.0281 - tn: 150578.0000 - auc: 0.6220 - prc: 0.1652 - val_loss: 0.4613 - val_Sensitivity: 0.0201 - val_tn: 38156.0000 - val_auc: 0.6777 - val_prc: 0.2457
Epoch 2/200
335/335 [==============================] - 2s 5ms/step - loss: 0.3279 - Sensitivity: 0.0210 - tn: 152111.0000 - auc: 0.6724 - prc: 0.2180 - val_loss: 0.3167 - val_Sensitivity: 0.0405 - val_tn: 38182.0000 - val_auc: 0.7035 - val_prc: 0.2918
Epoch 3/200
335/335 [==============================] - 2s 5ms/step - loss: 0.3210 - Sensitivity: 0.0413 - tn: 152129.0000 - auc: 0.6860 - prc: 0.2565 - val_loss: 0.3101 - val_Sensitivity: 0.0633 - val_tn: 38167.0000 - val_auc: 0.7159 - val_prc: 0.3140
Epoch 4/200
335/335 [==============================] - 2s 5ms/step - loss: 0.3154 - Sensitivity: 0.0608 - tn: 152149.0000 - auc: 0.6949 - prc: 0.2844 - val_loss: 0.3056 - val_Sensitivity: 0.0819 - val_tn: 38129.0000 - val_auc: 0.7161 - val_prc:

In [6]:
#fit and calculate performance metrics for RF and XGB
rf = RandomForestClassifier(n_estimators=1000, min_samples_split=2, min_samples_leaf=4, max_features='sqrt', max_depth=20, bootstrap=False)
rf.fit(X, y)
rf_preds_readm = rf.predict_proba(X_test)[:,1]
rf_auc_readm = roc_auc_ci(y_test, rf_preds_readm)
rf_prc_readm = roc_prc_ci(y_test, rf_preds_readm)
xgb = XGBClassifier(subsample=0.6, n_estimators=100, min_child_weight=10, max_depth=5, learning_rate=0.05, colsample_bytree=0.8)
xgb.fit(X, y)
xgb_preds_readm = xgb.predict_proba(X_test)[:,1]
xgb_auc_readm = roc_auc_ci(y_test, xgb_preds_readm)
xgb_prc_readm = roc_prc_ci(y_test, xgb_preds_readm)


KeyboardInterrupt: 

In [7]:
#fit and calculate performance metrics for LR
lr = LogisticRegression(max_iter=1000, penalty='none')
lr.fit(X, y)
lr_preds_readm = lr.predict_proba(X_test)[:,1]
lr_auc_readm = roc_auc_ci(y_test, lr_preds_readm)
lr_prc_readm = roc_prc_ci(y_test, lr_preds_readm)
lr_auc_readm

/home/kchen/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.6761935377159206, 0.6844288993941394, 0.6926642610723582)

In [ ]:
#get fpr/tpr and precision/recall for curves
lr_fpr_readm, lr_tpr_readm, _ = roc_curve(y_test, lr_preds_readm)
%store lr_fpr_readm
%store lr_tpr_readm
lr_prec_readm, lr_rec_readm, _ = precision_recall_curve(y_test, lr_preds_readm)
%store lr_prec_readm
%store lr_rec_readm

Stored 'lr_fpr_readm' (ndarray)
Stored 'lr_tpr_readm' (ndarray)
Stored 'lr_prec_readm' (ndarray)
Stored 'lr_rec_readm' (ndarray)


In [ ]:
rf_fpr_readm, rf_tpr_readm, _ = roc_curve(y_test, rf_preds_readm)
%store rf_fpr_readm
%store rf_tpr_readm
rf_prec_readm, rf_rec_readm, _ = precision_recall_curve(y_test, rf_preds_readm)
%store rf_prec_readm
%store rf_rec_readm

Stored 'rf_fpr_readm' (ndarray)
Stored 'rf_tpr_readm' (ndarray)
Stored 'rf_prec_readm' (ndarray)
Stored 'rf_rec_readm' (ndarray)


In [ ]:
xgb_fpr_readm, xgb_tpr_readm, _ = roc_curve(y_test, xgb_preds_readm)
%store xgb_fpr_readm
%store xgb_tpr_readm
xgb_prec_readm, xgb_rec_readm, _ = precision_recall_curve(y_test, xgb_preds_readm)
%store xgb_prec_readm
%store xgb_rec_readm

Stored 'xgb_fpr_readm' (ndarray)
Stored 'xgb_tpr_readm' (ndarray)
Stored 'xgb_prec_readm' (ndarray)
Stored 'xgb_rec_readm' (ndarray)


In [ ]:
#write results to txt file
with open('readm_results.txt', 'w') as f:
    f.write('AUROC\n')
    f.write('LR: '+str(round(lr_auc_readm[1], 3))+' (95% CI'+str(round(lr_auc_readm[0], 3))+'-'+str(round(lr_auc_readm[2], 3))+')\n')
    f.write('RF: '+str(round(rf_auc_readm[1], 3))+' (95% CI'+str(round(rf_auc_readm[0], 3))+'-'+str(round(rf_auc_readm[2], 3))+')\n')
    f.write('XGB: '+str(round(xgb_auc_readm[1], 3))+' (95% CI'+str(round(xgb_auc_readm[0], 3))+'-'+str(round(xgb_auc_readm[2], 3))+')\n')
    f.write('NN: '+str(round(ann_auc_readm[1], 3))+' (95% CI'+str(round(ann_auc_readm[0], 3))+'-'+str(round(ann_auc_readm[2], 3))+')\n')
    f.write('AUPRC\n')
    f.write('LR: '+str(round(lr_prc_readm[1], 3))+' (95% CI'+str(round(lr_prc_readm[0], 3))+'-'+str(round(lr_prc_readm[2], 3))+')\n')
    f.write('RF: '+str(round(rf_prc_readm[1], 3))+' (95% CI'+str(round(rf_prc_readm[0], 3))+'-'+str(round(rf_prc_readm[2], 3))+')\n')
    f.write('XGB: '+str(round(xgb_prc_readm[1], 3))+' (95% CI'+str(round(xgb_prc_readm[0], 3))+'-'+str(round(xgb_prc_readm[2], 3))+')\n')
    f.write('NN: '+str(round(ann_prc_readm[1], 3))+' (95% CI'+str(round(ann_prc_readm[0], 3))+'-'+str(round(ann_prc_readm[2], 3))+')\n')
#write results to csv file
with open('readm_results.csv', 'w', newline='') as csvfile:
    fieldnames = ['model','AUROC mean', 'AUROC 95% CI', 'AUPRC mean', 'AUPRC 95% CI']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'model':'Logistic Regression', 'AUROC mean':round(lr_auc_readm[1], 3), 'AUROC 95% CI':str(round(lr_auc_readm[0], 3))+'-'+str(round(lr_auc_readm[2], 3)), 'AUPRC mean':round(lr_prc_readm[1], 3), 'AUPRC 95% CI':str(round(lr_prc_readm[0], 3))+'-'+str(round(lr_prc_readm[2], 3))})
    writer.writerow({'model':'Random Forest', 'AUROC mean':round(rf_auc_readm[1], 3), 'AUROC 95% CI':str(round(rf_auc_readm[0], 3))+'-'+str(round(rf_auc_readm[2], 3)), 'AUPRC mean':round(rf_prc_readm[1], 3), 'AUPRC 95% CI':str(round(rf_prc_readm[0], 3))+'-'+str(round(rf_prc_readm[2], 3))})
    writer.writerow({'model':'XGBoost', 'AUROC mean':round(xgb_auc_readm[1], 3), 'AUROC 95% CI':str(round(xgb_auc_readm[0], 3))+'-'+str(round(xgb_auc_readm[2], 3)), 'AUPRC mean':round(xgb_prc_readm[1], 3), 'AUPRC 95% CI':str(round(xgb_prc_readm[0], 3))+'-'+str(round(xgb_prc_readm[2], 3))})
    writer.writerow({'model':'Neural Network', 'AUROC mean':round(ann_auc_readm[1], 3), 'AUROC 95% CI':str(round(ann_auc_readm[0], 3))+'-'+str(round(ann_auc_readm[2], 3)), 'AUPRC mean':round(ann_prc_readm[1], 3), 'AUPRC 95% CI':str(round(ann_prc_readm[0], 3))+'-'+str(round(ann_prc_readm[2], 3))})

In [ ]:
#get p-value for ann vs lr using Delong method from https://biasedml.com/roc-comparison/
%store -r ann_preds_readm
%store -r lr_preds_readm
from matplotlib import pyplot as plt
import scipy.stats as st
from sklearn import metrics


no stored variable or alias ann_preds_readm
no stored variable or alias lr_preds_readm


In [8]:

def auc(X, Y):
    return 1/(len(X)*len(Y)) * sum([kernel(x, y) for x in X for y in Y])
def kernel(X, Y):
    return .5 if Y==X else int(Y < X)
def structural_components(X, Y):
    V10 = [1/len(Y) * sum([kernel(x, y) for y in Y]) for x in X]
    V01 = [1/len(X) * sum([kernel(x, y) for x in X]) for y in Y]
    return V10, V01
    

def get_S_entry(V_A, V_B, auc_A, auc_B):
    return 1/(len(V_A)-1) * sum([(a-auc_A)*(b-auc_B) for a,b in zip(V_A, V_B)])
def z_score(var_A, var_B, covar_AB, auc_A, auc_B):
    return (auc_A - auc_B)/((var_A + var_B - 2*covar_AB)**(.5))


# Model A (random) vs. "good" model B

preds_A = ann_preds_readm
preds_B = lr_preds_readm
actual = y_test

actual = actual.array

def group_preds_by_label(preds, actual):
    X = [p for (p, a) in zip(preds, actual) if a]
    Y = [p for (p, a) in zip(preds, actual) if not a]
    return X, Y


X_A, Y_A = group_preds_by_label(preds_A, actual)
X_B, Y_B = group_preds_by_label(preds_B, actual)
V_A10, V_A01 = structural_components(X_A, Y_A)
V_B10, V_B01 = structural_components(X_B, Y_B)
auc_A = auc(X_A, Y_A)
auc_B = auc(X_B, Y_B)


# Compute entries of covariance matrix S (covar_AB = covar_BA)
var_A = (get_S_entry(V_A10, V_A10, auc_A, auc_A) * 1/len(V_A10)
        + get_S_entry(V_A01, V_A01, auc_A, auc_A) * 1/len(V_A01))
var_B = (get_S_entry(V_B10, V_B10, auc_B, auc_B) * 1/len(V_B10)
        + get_S_entry(V_B01, V_B01, auc_B, auc_B) * 1/len(V_B01))
covar_AB = (get_S_entry(V_A10, V_B10, auc_A, auc_B) * 1/len(V_A10)
            + get_S_entry(V_A01, V_B01, auc_A, auc_B) * 1/len(V_A01))

# Two tailed test
z = z_score(var_A, var_B, covar_AB, auc_A, auc_B)

p = st.norm.sf(abs(z))*2
p

NameError: name 'x' is not defined

In [49]:
#find the sensitivity and specificity of the NN model at thresholds
from sklearn.metrics import recall_score
from imblearn.metrics import specificity_score

thresh = np.arange(0, 1, 0.0001)
#calculate sensitivity at thresholds
lr_sens = {}
for t in thresh:
    lr_sens[t] = recall_score(y_test, lr_preds_readm > t)
lr_spec = {}
for t in thresh:
    lr_spec[t] = specificity_score(y_test, lr_preds_readm > t)
ann_sens = {}
for t in thresh:
    ann_sens[t] = recall_score(y_test, ann_preds_readm > t)
ann_spec = {}
for t in thresh:
    ann_spec[t] = specificity_score(y_test, ann_preds_readm > t)
def get_senspec(thresh):
    print(lr_sens[thresh], lr_spec[thresh])
    print(ann_sens[thresh], ann_spec[thresh])


In [75]:
#find the sensitivity at various thresholds of specificity
def get_lrsenspec(thresh):
    print(recall_score(y_test, lr_preds_readm > thresh), specificity_score(y_test, lr_preds_readm > thresh))
def get_annsenspec(thresh):
    print(recall_score(y_test, ann_preds_readm > thresh), specificity_score(y_test, ann_preds_readm > thresh))

In [74]:
get_lrsenspec(0.0609)

0.9671669793621013 0.10048637239607232


In [81]:
get_annsenspec(0.02897)

0.9778611632270169 0.10048637239607232


In [90]:
get_lrsenspec(0.0748)

0.8855534709193246 0.3007708543635863


In [107]:
get_annsenspec(0.0414)

0.9185741088180113 0.3000137652564926


In [120]:
get_lrsenspec(0.08962)

0.7675422138836773 0.5000458841883088


In [142]:
get_annsenspec(0.06106)

0.8181988742964352 0.5000229420941543


In [153]:
get_lrsenspec(0.1105)

0.5784240150093809 0.7003762503441314


In [166]:
get_annsenspec(0.1068)

0.6652908067542214 0.700674497568138


In [177]:
get_lrsenspec(0.164)

0.2373358348968105 0.9007066164999541


In [188]:
get_annsenspec(0.1839)

0.39849906191369605 0.9008213269707259
